In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import os
import torch.nn.functional as F

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root, train=True, val=False, transform=None):
        self.root = root
        self.transform = transform
        self.images = []
        self.labels = []

        # Load images and labels from your dataset
        if train:
            folder = os.path.join(root, 'train')
        elif val:
            folder = os.path.join(root, 'val')
        else:
            folder = os.path.join(root, 'test')

        for class_folder in os.listdir(folder):
            class_folder_path = os.path.join(folder, class_folder)
            for filename in os.listdir(class_folder_path):
                img_path = os.path.join(class_folder_path, filename)
                label = 0 if class_folder == 'no-snow' else 1
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)

        return img, label

In [3]:
# Define the data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        # Define pooling layers
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        # Define 1x1 convolutional layer
        self.conv = nn.Conv2d(in_channels * 2, 1, kernel_size=1)
        
        # Sigmoid activation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Compute average and max pooled representations
        avg_out = self.avg_pool(x)
        max_out = self.max_pool(x)
        
        # Concatenate average and max pooled representations
        pooled_features = torch.cat((avg_out, max_out), dim=1)
        
        # Apply 1x1 convolutional layer
        conv_out = self.conv(pooled_features)
        
        # Apply sigmoid activation
        attn_scores = self.sigmoid(conv_out)

        # Apply attention weights to the input features
        attn_output = x * attn_scores

        return attn_output, attn_scores



# Define the architecture of the model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.spatial_attention = SpatialAttention(in_channels=64)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))

        # Apply spatial attention
        x, _ = self.spatial_attention(x)

        x = x.view(-1, 64 * 8 * 8)  # Reshape to (batch, features)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Instantiate your model
model = CustomModel().cuda()

# Load the saved model parameters
model.load_state_dict(torch.load('../../models/05-25-2024/best_model.pt'))

<All keys matched successfully>

In [6]:
def f2_score(true_labels, predictions):
    true_positives = sum((tl == 1) and (p == 1) for tl, p in zip(true_labels, predictions))
    false_positives = sum((tl == 0) and (p == 1) for tl, p in zip(true_labels, predictions))
    false_negatives = sum((tl == 1) and (p == 0) for tl, p in zip(true_labels, predictions))

    beta = 2  # Weight assigned to recall
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    if precision == 0 or recall == 0:
        return 0
    
    f2 = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    return f2

In [7]:
test_dataset = CustomDataset(root='..\\..\\AI_generated_Data\\', train=False, val=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Predict the classes for the test dataset
predicted_classes = []
with torch.no_grad():
    for data, target in test_loader:
        output = model(data.cuda())
        _, predicted = torch.max(output.data, 1)
        predicted_classes.extend(predicted.tolist())

# Create a list of labels from the test dataset
y_true = []
with torch.no_grad():
    for _, target in test_loader:
        y_true.extend(target.tolist())

# Calculate different metrics
accuracy = accuracy_score(y_true, predicted_classes)
precision = precision_score(y_true, predicted_classes, average='weighted')
recall = recall_score(y_true, predicted_classes, average='weighted')
f2 = f2_score(y_true, predicted_classes)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F2-score:', f2)

Accuracy: 0.8578199052132701
Precision: 0.8602375424525587
Recall: 0.8578199052132701
F2-score: 0.9076175040518638


In [9]:
print(predicted_classes)
print(y_true)

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,